In [ ]:
! python train.py --root-imgs /home/jovyan/work/cob-test/pascal-voc/VOC2012 --root-segs /home/jovyan/work/cob-test/trainval --run-path /home/jovyan/work/cob-test/runs/cob --cuda

In [ ]:
import scipy.io as io
lbl_path = "trainval/2008_000002.mat"
data = io.loadmat(lbl_path)
lbl = data['LabelMap']

In [ ]:
lbl

In [ ]:
import numpy as np
from utils.dataloader import interpolate_to_polygon

In [ ]:
image = interpolate_to_polygon(lbl).astype('float64')
image *= 255.0/image.max()

In [ ]:
from PIL import Image
img = Image.fromarray(np.uint8(image * 255) , 'L')

In [ ]:
img

In [ ]:
interpolate_to_polygon(lbl).astype(np.uint8)

In [ ]:
import torch
from torch.utils.data import DataLoader
from utils.dataloader import CobDataLoader

In [ ]:
dset_val = CobDataLoader(root_imgs="/home/jovyan/work/cob-test/pascal-voc/VOC2012", root_segs="/home/jovyan/work/cob-test/trainval", split='val')
dl_val = DataLoader(dset_val, collate_fn=dset_val.collate_fn, batch_size=1)

In [ ]:
from models.cobnet import CobNet
model = CobNet()
model.load_state_dict(torch.load("runs/cob/checkpoints/cp_or.pth.tar"))

In [ ]:
batch = next(iter(dl_val))
batch = batch["image"]

In [ ]:
from skimage.io import imread
from imgaug import augmenters as iaa
from utils.augmenters import Normalize, rescale_augmenter

normalization_mean=[0.485, 0.456, 0.406]
normalization_std=[0.229, 0.224, 0.225]

aug = iaa.Sequential([
    iaa.size.Resize(250),
    iaa.Flipud(p=0.5),
    iaa.Fliplr(p=.5),
    iaa.Fliplr(p=.5),
    iaa.Rotate([360 / 4 * i for i in range(4)]),
    rescale_augmenter,
    Normalize(mean=normalization_mean, std=normalization_std)
])
aug_det = aug.to_deterministic()

In [ ]:
image = imread("pascal-voc/VOC2012/JPEGImages/2007_000033.jpg")
image = image[np.newaxis, ...]
#image_t = np.stack((image[:,:,0], image[:,:,1], image[:,:,2]), axis = 0)
image2 = imread("denmark/1.jpg")
image2 = image2[np.newaxis, ...]
#image2_t = np.stack((image2[:,:,0], image2[:,:,1], image2[:,:,2]), axis = 0)

sample = aug_det(images=image2)

In [ ]:
csample = sample[0]
in_img = np.stack((csample[:,:,0], csample[:,:,1], csample[:,:,2]), axis = 0)
in_img = in_img[np.newaxis, ...]
model.eval()
with torch.no_grad():
    res = model(torch.tensor(in_img).float())

In [ ]:
from torchvision.utils import make_grid, save_image
n_orient=4

fine = res['y_fine'].sigmoid()
coarse = res['y_coarse'].sigmoid()

total_orients = len(res['orientations'])
orients_idx = np.arange(0, total_orients, step=total_orients // n_orient)
orients = []
for i in orients_idx:
    o_ = make_grid(res['orientations'][i].sigmoid(), nrow=1)
    orients.append(o_)

save_image(fine, 'test.png')
save_image(coarse, 'coarse.png')
save_image([*orients], 'orients.png')

In [ ]:
img = Image.fromarray(image2[0] , 'RGB')
img

In [ ]:
torch.cuda.is_available()

In [ ]:
import helpers.cob.cobtrain as cobtrain

In [ ]:
cfg = {
    'images': "/home/jovyan/work/ma/helpers/cob/pascal-voc/VOC2012",
    'segments': "/home/jovyan/work/ma/helpers/cob/trainval",
    'run': "/home/jovyan/work/runs/cob",
    'lr': 1e-4,
    'decay': 2e-4,
    'momentum': 0.9,
    'epochs-div-lr': 6,
    'epochs': 10,
    'aug-n-angles': 4,
}

In [ ]:
cobtrain.initRetrain(cfg)

In [1]:
import torch
points1 = torch.load("/home/jovyan/work/ma/points1.pt")
probs1 = torch.load("/home/jovyan/work/ma/probs1.pt")
points16 = torch.load("/home/jovyan/work/ma/points16.pt")
probs16 = torch.load("/home/jovyan/work/ma/probs16.pt")

In [2]:
from models.losses import lcfcn_loss
#loss2 = 0.
#for i in range(len(probs16)):
loss = lcfcn_loss.computeLoss(points = points16, probs = probs16)
print(loss)

torch.Size([1000000])
57
tensor(63.7695, device='cuda:0', grad_fn=<AddBackward0>)


In [ ]:
import torch
from torch import nn

def getBack(var_grad_fn):
    print(var_grad_fn)
    for n in var_grad_fn.next_functions:
        if n[0]:
            try:
                tensor = getattr(n[0], 'variable')
                print(n[0])
                print('Tensor with grad found:', tensor)
                print(' - gradient:', tensor.grad)
                print()
            except AttributeError as e:
                getBack(n[0])


getBack(loss.grad_fn)

In [ ]:
import torch
from models.losses import lcfcn_loss

In [ ]:
points16 = torch.load("/home/jovyan/work/ma/points16.pt")
probs16 = torch.load("/home/jovyan/work/ma/probs16.pt")

checklist = [2.0794, 9.0109, 3.4657, 2.0794, 7.6246, 4.8520, 2.0794, 2.0794, 4.8520, 7.6246, 3.4657, 2.0794, 3.4657, 2.0794, 4.8520, 2.0794]
failed = False

for i in range(len(probs16)):
    loss = round(lcfcn_loss.computeLoss(points = points16[i].unsqueeze(0), probs = probs16[i].unsqueeze(0)).item(), 4)
    if loss != checklist[i]:
        failed = True
        print(f"Failed i: {i} with loss: {loss} - should be {checklist[i]}")

if failed == False: print("Success!")

In [ ]:
a = torch.where(probs1.squeeze().view(-1) == 0.5)[0]

In [ ]:
a.add(255)